In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

In [ ]:
message = model.invoke(prompt_value)
message

In [2]:
# Requires:
# pip install langchain docarray tiktoken

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho when he was 18", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [9]:
(setup_and_retrieval|prompt|model).invoke("where did harrison work?")

AIMessage(content='Harrison worked at Kensho.')

In [10]:
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

"Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't find its scoop!"

In [11]:
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream go to therapy?

Because it had a rocky road!

In [12]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream go to therapy?\n\nBecause it was feeling a little rocky road!',
 'Why did the spaghetti go to the party? Because it was pasta-tively saucy!',
 'Why did the dumpling go to the gym?\n\nBecause it wanted to get a little more "steam" in its buns!']

In [13]:
chain.ainvoke("ice cream")

<coroutine object RunnableSequence.ainvoke at 0x7f6a7491b610>

In [14]:
from langchain.chat_models import ChatAnthropic

anthropic = ChatAnthropic(model="claude-2")
anthropic_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | anthropic
    | output_parser
)

anthropic_chain.invoke("ice cream")

ValidationError: 1 validation error for ChatAnthropic
__root__
  Did not find anthropic_api_key, please add an environment variable `ANTHROPIC_API_KEY` which contains it, or pass  `anthropic_api_key` as a named parameter. (type=value_error)

In [1]:
%pip install --upgrade --quiet  duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
ev =search.invoke("thumbnail ctr", num_results=10)
ev

In [29]:
import json

print(json.loads(ev))

TypeError: the JSON object must be str, bytes or bytearray, not coroutine